In [13]:
############### -------- configure patch size and copy the pathces tuplte to the next cell for computing 1-d averaging for all frames ------------- ###################
%matplotlib widget
import os, shutil, h5py, time
import numpy as np

# specs
frame = 1348
source_file = '1934_SN.h5'                                         

# xy,   width_p, height_p, bounding_box_color for drawing Rectangle
patches = [

                          ] 

## semi-specs
valid_range_min, valid_range_max = (0,10)
scattering = '_WAXS2'
masked_file = f'{source_file.split(".")[0]}_masked_{frame}.h5'

## computation
with h5py.File(source_file,'r') as hdf:

    print('data is being loaded ...')
    dset = hdf.get(f'{masked_file.split("_masked")[0]}/primary/data')
    print('data is loaded ...')
    
    print('frame information extraction starts ...')
    dset_saxs = np.expand_dims(dset['pil1M_image'][frame], axis=0)
    dset_waxs = np.expand_dims(dset['pilW2_image'][frame], axis=0)
    print(f'frame information extraction completes with _SAXS shape {dset_saxs.shape} _WAXS shape {dset_waxs.shape}...')

if os.path.isfile(masked_file):
    os.remove(masked_file)
with h5py.File(masked_file,'w') as hdf:

    for frame in range(len(dset_waxs)):
        for xy, width_p, height_p, color in patches:           
            ##### careful here dset_waxs = 0 ; becasue there is only one frame 
            dset_waxs[0][xy[1]: xy[1]+height_p, xy[0]: xy[0]+width_p] = 0   # height px and width px will not be counted img[0:70, 860:860+60]

    dset = hdf.create_group(f'/{masked_file.split("_masked")[0]}/primary/data')
    dset.create_dataset('pil1M_image', data = dset_saxs, compression="lzf")
    dset.create_dataset('pilW2_image', data = dset_waxs, compression="lzf")
    
        
from py4xs.hdf import h5xs,h5exp,lsh5
from py4xs.data2d import Data2d
import numpy as np
import pylab as plt
de = h5exp("exp.h5")
qgrid2 = np.hstack([np.arange(0.005, 0.0499, 0.001), np.arange(0.05, 0.099, 0.002), np.arange(0.1, 3.2, 0.005)])

dt  = h5xs(masked_file, [de.detectors, qgrid2])
dt.load_data(N=1, debug=True)
#dt.d1s[f'{masked_file.split("_masked")[0]}']['_WAXS2'][0].plot()

# figure with patches
from matplotlib.patches import Rectangle

img = np.clip(dset_waxs[0], valid_range_min, valid_range_max, dtype = np.int8)

plt.figure()
plt.imshow(img, cmap = 'rainbow')
for xy, width_p, height_p, color in patches:
    plt.gca().add_patch(Rectangle(xy=xy, width=width_p, height=height_p,
                                angle=0,
                                edgecolor=color,
                                facecolor='none',
                                fill=False,
                                lw=1))
    
    img[xy[1]: xy[1]+height_p, xy[0]: xy[0]+width_p] = 0   # height px and width px will not be counted img[0:70, 860:860+60]
plt.show()

### plot WAXS data
with h5py.File(masked_file,'r') as hdf:
    dset_merged = hdf.get(f'{masked_file.split("_masked")[0]}/processed')
    dset_merged = dset_merged[scattering][:]

f, ax = plt.subplots(num=f'scattering data')
plt.subplots_adjust(bottom=0.15)
#ax = plt.gca()
ax.set_xlabel("$q (\AA^{-1})$",)
ax.set_ylabel("$I$",)
ax.set_xscale('linear')
ax.set_yscale('linear')
#ax.errorbar(qgrid2, dset_merged[0][0], dset_merged[0][1], label=f'{scattering} {frame}')   # here dataset has only one frame
ax.plot(qgrid2, dset_merged[0][0], label=f'{scattering} {frame}')   # here dataset has only one frame
ax.legend()
plt.show()

if os.path.isfile(masked_file):
    os.remove(masked_file)

data is being loaded ...
data is loaded ...
frame information extraction starts ...
frame information extraction completes with _SAXS shape (1, 1043, 981) _WAXS shape (1, 1043, 981)...
start processing: load_data()
['1934_SN']
processing started: sample = 1934_SN, starting frame = #0
loading data from 2D image:  1934_SN_f00000_SAXS
loading data from 2D image:  1934_SN_f00000_WAXS2
processing completed:  1934_SN 0
writting attribute to 1934_SN: merged
writting attribute to 1934_SN: _SAXS
writting attribute to 1934_SN: _WAXS2
done, time lapsed: 0.46 sec


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [4]:
# 1-d averaging after patches for one file
from essential_func import *

### spec
source_file = "2048_B8.h5"
patches = [
            [ [800,  0], 180,  150, "red"   ], 
            [ [195, 0], 150,   125, "green" ], 
            [ [0,  225], 35,   35, "orange"], 
            [ [580,215], 40,   50, "blue"  ],
            [ [875,360], 55,   50, "pink"  ],
            [ [0,  560], 120,  110, "purple"], 
            [ [315,740], 60,   65, "black" ],
            [ [625,890], 30,   30, "yellow"],
            [ [0,935],   75,   55, "blue"],
            [ [865,1005],115,  35, "green"]
          ]

### computation
masked_file = circ_avg_from_patches(source_file, patches)
print(f'Successfully created (source file, masked file)')

2048_B8_masked.h5 is being created in /Users/bashit.a/Documents/Alzheimer/Dec-15 ...
2048_B8_masked.h5 copy is done
2048_B8_masked.h5 Loading data into a numpy array started 
2048_B8_masked.h5 Loading data into a numpy array finished in 64.62004645899998 seconds
2048_B8_masked.h5 Patching Started 
2048_B8_masked.h5 Patching Finished
2048_B8_masked.h5 file saving staring ... 
2048_B8_masked.h5 file saving finished in 108.67543654100001 seconds
Circular averaging staring now ... 
data received: sn=2048_B8, fr1=0
data received: sn=2048_B8, fr1=617
data received: sn=2048_B8, fr1=1234
data received: sn=2048_B8, fr1=1851
data received: sn=2048_B8, fr1=2468
data received: sn=2048_B8, fr1=3085
data received: sn=2048_B8, fr1=3702
data received: sn=2048_B8, fr1=4319
2048_B8_masked.h5total 1-d averaging time 552.066487583 seconds
Successfully created (source file, masked file) 




In [1]:
# 1-d averaging after patches for ALL files in the current directory
from essential_func import *

data = get_json_str_data("config.json")
### do masking and 1-d averaging (Lin Yang's code by BNL used here for 1-d averaging)
files = []
for file in data['files']:
    source_file , patches = file['name'], file['patches']
    try: masked_file = circ_avg_from_patches(source_file, patches)
    except: continue
    files.append((source_file, masked_file))


print(f'Successfully created (source file, masked file) \n\n')
[print(s, '--->' ,t) for s,t in files]
pass

1934_B8_masked.h5 is being created in /Users/bashit.a/Documents/Alzheimer/Dec-15 ...
1934_B8_masked.h5 copy is done
1934_B8_masked.h5 Loading data into a numpy array started 
1934_B8_masked.h5 Loading data into a numpy array finished in 50.873808 seconds
1934_B8_masked.h5 Patching Started 
1934_B8_masked.h5 Patching Finished
1934_B8_masked.h5 file saving staring ... 
1934_B8_masked.h5 file saving finished in 91.344045 seconds
Circular averaging staring now ... 
data received: sn=1934_B8, fr1=0
data received: sn=1934_B8, fr1=517
data received: sn=1934_B8, fr1=1034
data received: sn=1934_B8, fr1=1551
data received: sn=1934_B8, fr1=2068
data received: sn=1934_B8, fr1=2585
data received: sn=1934_B8, fr1=3102
data received: sn=1934_B8, fr1=3619
1934_B8_masked.h5total 1-d averaging time 455.49015299999996 seconds
1934_SN_masked.h5 is being created in /Users/bashit.a/Documents/Alzheimer/Dec-15 ...
1934_SN_masked.h5 copy is done
1934_SN_masked.h5 Loading data into a numpy array started 


KeyboardInterrupt: 

In [14]:
### 1d circularly averaged loading for masked file 
%matplotlib widget
from essential_func import *
import ipywidgets   # from IPython.display import display; display(amp)

# specs  
masked_file = "1934_SN.h5"   
scattering = '_WAXS2'

# semi-specs
file = masked_file 
qgrid2 = np.hstack([np.arange(0.005, 0.0499, 0.001), np.arange(0.05, 0.099, 0.002), np.arange(0.1, 3.2, 0.005)])

# computation
Iq = read_Iq(file, scattering)    
Width, Height = width_height(file)    
n_patterns = Width*Height 

f,ax = plt.subplots()
def update_plot(frame):
    ax.clear()
    #ax.set_yscale('log')
    ax.plot(qgrid2, (Iq[frame]), label = f'{scattering} {frame}')

frame = ipywidgets.IntSlider(min=0, max=n_patterns-1, value=0, description = "Frame")
ipywidgets.interactive(update_plot, frame=frame)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(IntSlider(value=0, description='Frame', max=4960), Output()), _dom_classes=('widget-inte…

In [5]:
### show heat map for any file
%matplotlib widget    
from essential_func import *

file = '2048_B8.h5'
#file = masked_file    
scatterings = ('_SAXS', '_WAXS2')

#plot_one_heat_maps(file, scatterings)
f = plot_heat_map_from_file(file, qgrid2, scatterings = scatterings)
print('Task Finished. Figure Number = ' , f.number)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

low valid idx = 2, low valid Q = 0.007, high valid idx = 124 , high valid Q = 0.370
low valid idx = 109, low valid Q = 0.295, high valid idx = 578 , high valid Q = 2.640
Task Finished. Figure Number =  1


In [1]:
################# --------- Find Q range to plot heat maps --------- #################
%matplotlib widget
from essential_func import *
import ipywidgets         # from IPython.display import display; display(amp)
from ipywidgets import fixed


# semi-specs
qgrid2 = np.hstack([np.arange(0.005, 0.0499, 0.001), np.arange(0.05, 0.099, 0.002), np.arange(0.1, 3.2, 0.005)])

### q_low, q_high, q_low - idx[2] and q_high + idx[2] for poly fit, number of points
indices = ((0.115, 0.16, 13,   48, '_SAXS' , 'bumpy'),      # 72, 81
           (1.48,  1.57, 10, 4.15, '_WAXS2', 'paraffin'),   # 345, 355
           (1.30,  1.38, 10,  4.7, '_WAXS2', 'amyloid'),    # 309, 323
           (1.89,  1.93,  7,  3.2, '_WAXS2', 'mica')        # 427, 433
          )

## polyhit with heatmap from drop down list
files = cwd_files_type_search('.h5')
dropdown = ipywidgets.Dropdown(options=files, value = files[0],description='File : ', disabled=False)
ipywidgets.interact(file_polyfit_heatmap_plot, file = dropdown, indices = fixed(indices), qgrid2 = fixed(qgrid2))


## polyhit with heatmap for one file
# file = '2048_B8.h5'
# file_polyfit_heatmap_plot(file, indices, qgrid2)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(IntSlider(value=0, continuous_update=False, description='Frame', max=4940), IntSlider(va…

interactive(children=(Button(description='Save PDF', style=ButtonStyle()), Output()), _dom_classes=('widget-in…

In [2]:
%matplotlib widget
from essential_func import *
from py4xs.hdf import h5xs
import os
from py4xs.slnxs import trans_mode

print(f'{os.getcwd()}')

# specs
# file = '1934_SN_masked.h5'  
# frames = (4518, 3687, 3045)     # amyloid, mica, dark  ---> frames can be choosen from SAXS heat map

file = '2048_B16.h5'
frames = (1346, 1358, 2616)     # amyloid, mica, dark
show_eb = False                  # show errorbar in the plot


## semi-spec
qgrid2 = np.hstack([np.arange(0.005, 0.0499, 0.001), np.arange(0.05, 0.099, 0.002), np.arange(0.1, 3.2, 0.005)])

# creating objects
dt  = h5xs(f'{file}', transField="em2_sum_all_mean_value")
sn = h5_top_group(file)
dt.load_d1s(sn=sn)

## heat map from file
scatterings = ('_SAXS', '_WAXS2')
f = plot_heat_map_from_file(file, qgrid2, scatterings = scatterings)

### 1-d plot for bright, moderate and dark frames
scatterings = ('_SAXS',          '_WAXS2',        'merged',    )
colors =      (('r','b','g'), ('r','b','g'), ('r+','bo','g--'),)
f, axs = plt.subplots(1, 3, figsize=(15,5), num=f'{file}')
xscale = 'linear'
yscale = 'log'

for i, (scattering, color) in enumerate(zip(scatterings,colors)):
    axs[i].plot(dt.qgrid, dt.d1s[sn][scattering][frames[0]].data, color[0], label = f'bright {frames[0]}')
    axs[i].plot(dt.qgrid, dt.d1s[sn][scattering][frames[1]].data, color[1], label = f'moderate {frames[1]}')
    axs[i].plot(dt.qgrid, dt.d1s[sn][scattering][frames[2]].data, color[2], label = f'dark {frames[2]}')
    axs[i].set(title = f'{scattering} {file}', xscale= xscale, yscale= yscale)
    axs[i].legend()

## background correction calculations
sc_factor=1.

dset = dt.d1s[sn][scattering][frames[0]]
dbak = dt.d1s[sn][scattering][frames[2]]

dt.set_trans(transMode=trans_mode.external)
sc  = dset.trans / dbak.trans
print(f'Scaling factor, {dset.trans}/{dbak.trans} = {sc}')
Iq  = dset.data - dbak.data * sc * sc_factor
dI  = dset.err  + dbak.err * sc * sc_factor
idx = (dset.data > 0) & (dbak.data > 0)

## plotting settings
f, axs = plt.subplots(1,2,figsize=(16,4))
plt.subplots_adjust(bottom=0.15)

axs[0].plot(dbak.qgrid[idx], dset.data[idx], color[0], label = f'bright {frames[0]}')
axs[0].plot(dbak.qgrid[idx], dbak.data[idx], color[2], label = f'dark-bcg {frames[2]}')

if show_eb:
    axs[0].errorbar(dbak.qgrid[idx], Iq[idx], dI[idx])
    axs[1].errorbar(dbak.qgrid[idx], Iq[idx], dI[idx], label = f'{frames[0]} bcor')
else:
    axs[0].plot(dbak.qgrid[idx], Iq[idx], label = 'bcor')
    axs[1].plot(dbak.qgrid[idx], Iq[idx], label = f'{frames[0]} bcor')


axs[0].set(xlabel = "$q (\AA^{-1})$", ylabel = "$I$", title = f'Background correction {file} errorbar = {show_eb}', xscale = xscale, yscale = yscale, )
axs[1].set(xscale = xscale, yscale = yscale, title = f'Final Background Subtracted {file} errorbar = {show_eb}' , )
axs[0].legend()
axs[1].legend()

/Users/bashit.a/Documents/Alzheimer/Dec-15


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

low valid idx = 2, low valid Q = 0.007, high valid idx = 124 , high valid Q = 0.370
low valid idx = 109, low valid Q = 0.295, high valid idx = 578 , high valid Q = 2.640


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Scaling factor, 45244.968/45641.047999999995 = 0.9913218469479492


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [3]:
idx

array([False, False,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,

In [2]:
# pdf generation package all files in the cwd
%matplotlib widget
from essential_func import *

### heat plots for all files in a folder
scatterings = ('_SAXS', '_WAXS2')
qgrid2 = np.hstack([np.arange(0.005, 0.0499, 0.001), np.arange(0.05, 0.099, 0.002), np.arange(0.1, 3.2, 0.005)])

plot_all_heat_maps_cwd('output.pdf', qgrid2, scatterings)

Loading file 1934_B8.h5


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

low valid idx = 2, low valid Q = 0.007, high valid idx = 124 , high valid Q = 0.370
low valid idx = 109, low valid Q = 0.295, high valid idx = 578 , high valid Q = 2.640
1934_B8.h5 Task Finished. Figure Number =  1
Loading file 1934_SN.h5


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

low valid idx = 2, low valid Q = 0.007, high valid idx = 124 , high valid Q = 0.370
low valid idx = 109, low valid Q = 0.295, high valid idx = 578 , high valid Q = 2.640
1934_SN.h5 Task Finished. Figure Number =  2
Loading file 1934_SNa.h5


...

Loading file mica.h5


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

low valid idx = 2, low valid Q = 0.007, high valid idx = 124 , high valid Q = 0.370
low valid idx = 109, low valid Q = 0.295, high valid idx = 578 , high valid Q = 2.640
mica.h5 Task Finished. Figure Number =  19
Loading file std.h5


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

PDF creation Finished


In [10]:
# import pylab as plt
# from matplotlib.widgets import Cursor
# import numpy as np
# x,  y  = np.random.randn(2,10)
# print(x.shape, y.shape)
# f, axs = plt.subplots(1,2)
# axs[0].plot(x,y,'o')
# axs[1].plot(x,y,'o')

# cursor = Cursor(axs[0],
#                 horizOn = True,
#                 vertOn = True,
#                 color= 'green',
#                 linewidth = 1
#                )
# pos = []
# def onclick(event):
#     x1, y1 = event.xdata, event.ydata
#     pos.append((x1,y1))
#     print(x1,y1)

# f.canvas.mpl_connect('button_press_event',  onclick)
#print(pos)

# def hover(event):
#     if event.inaxes != ax:
#         return
#     info = 'x={:.2f}, y={:.2f}'.format(event.xdata, event.ydata)
#     ax.format_coord = lambda x, y: info
# cid = ax.figure.canvas.mpl_connect("motion_notify_event", hover)

# ratio = Height/Width
# xleft, xright = ax.get_xlim()
# ybottom, ytop = ax.get_ylim()
# abs((xright-xleft)/(ybottom-ytop))*ratio
# 1/ax.get_data_ratio()

In [11]:
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np
# Fixing random state for reproducibility
np.random.seed(19680801)

Height = 3
Width = 6
X = np.random.randint(1, 20, size=(Height, Width))

fig, ax = plt.subplots()
ax.imshow(X, interpolation = 'none', origin='upper', extent=[0,Width,0,Height],aspect='equal')

def snaking( Width, Height, X=None,):
    if X is None:
        X_coor = np.arange(Height*Width).reshape(Height,Width)
    else:
        X_coor = X
    output = np.zeros_like(X_coor)           # 
    temp = np.flipud(X_coor)                # flipud matrix
    idx_odd = np.arange(1,Height,2)                # Get odd indices to fliplr
    output[idx_odd,:]  = np.fliplr(temp[idx_odd])   # flipping odd indices
    idx_even = np.arange(0,Height,2)               # Get odd indices to leave to as it is
    output[idx_even,:] = temp[idx_even]             # leaving as it is
    return output

frame_cor = snaking(Width, Height)
diff_patterns = snaking(Width, Height, X)

print('Diff_patt_idx_before snaking: \n', np.arange(Height*Width).reshape(Height,Width), '\n\n',
      'Diff_patt_before snaking: \n', X, '\n\n', 
      'X,Y coordinates_after_snaking: \n', frame_cor, '\n\n',
      'Diff_patt_after snaking: \n', diff_patterns, '\n\n',)

numrows, numcols = X.shape
def format_coord(x, y):
    col = int(x)
    row = int(y)
    if 0 <= col < numcols and 0 <= row < numrows:
        z = np.flipud(frame_cor)[row, col]
        return 'x=%1.4f, y=%1.4f, FRAME=%d' % (x, y, z)
    else:
        return 'x=%1.4f, y=%1.4f' % (x, y)

ax.format_coord = format_coord

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Diff_patt_idx_before snaking: 
 [[ 0  1  2  3  4  5]
 [ 6  7  8  9 10 11]
 [12 13 14 15 16 17]] 

 Diff_patt_before snaking: 
 [[13 14  7  3  3  4]
 [18  5 11 19 12  9]
 [ 6 12  4  6 17 10]] 

 X,Y coordinates_after_snaking: 
 [[12 13 14 15 16 17]
 [11 10  9  8  7  6]
 [ 0  1  2  3  4  5]] 

 Diff_patt_after snaking: 
 [[ 6 12  4  6 17 10]
 [ 9 12 19 11  5 18]
 [13 14  7  3  3  4]] 




In [ ]:
# --------- template matching purposes -------------


# plot zoomed patch figure
#plt.figure()
#pil_im = img[0:150, 800:800+150]
#plt.imshow(pil_im, cmap = 'rainbow', ); 
#plt.show();

# --------- template matching -------------
# template_file = ''
# Cropped image of above dimension OPENCV to PIL_IMAGE
# https://stackoverflow.com/questions/43232813/convert-opencv-image-format-to-pil-image-format
# PIL to OpenCV
#pil_im = Image.fromarray(img_rgb)
#pil_im = pil_im.crop((left, top, right, bottom))
# load template and find height and width of the template shape (convert your BGR image to Gray image)
#template = pil_im
#h, w = template.shape[::]
# print(f'template width {w} Height {h}')
# print(np.min(template), np.max(template), np.min(img), np.max(img))
# print(f'templete {template.shape} type {type(template)} pil_im {img.shape} type {type(img)}')
#res = cv2.matchTemplate(img_rgb, template, cv2.TM_CCOEFF)